In [1]:
from swyne.node import *
from swyne.layout import *
from swyne.focus import *
from swyne.images import *
from swyne.dialog import *
import pyglet
from pyglet.gl import *


sprites = load_spritesheet("pat_asteroids/pat_asteroids.json")

In [2]:
w = NodeWindow()
w.fps = 30 # I want on_frame events later.

w.node, keys = deserialize_node("""
BackgroundNode [255,255,255,255] # white background
-cols ColsLayoutNode # divide the page into columns
""")


"""
w.node.children[0]
# keys["cols"] refers the ColsLayoutNode

<div>
    <p> Text
    </p>
</div>

my_div = document.createElement("div")
my_p = document.createElement("p")
my_div.add_child(my_p)
my_p.text = "Text"

div
-p "More text"
-div
--my_paragraph p "Text"


<div>
 <p text="More text"></p>
 <div><p text="Text" id="my_paragraph"></p></div>
</div>

root_div.children[1].children[0].text = "changed text"
"""

'\nw.node.children[0]\n# keys["cols"] refers the ColsLayoutNode\n\n<div>\n    <p> Text\n    </p>\n</div>\n\nmy_div = document.createElement("div")\nmy_p = document.createElement("p")\nmy_div.add_child(my_p)\nmy_p.text = "Text"\n\ndiv\n-p "More text"\n-div\n--my_paragraph p "Text"\n\n\n<div>\n <p text="More text"></p>\n <div><p text="Text" id="my_paragraph"></p></div>\n</div>\n\nroot_div.children[1].children[0].text = "changed text"\n'

In [3]:

# first example: some rectangles
first_column, _ = deserialize_node("""
PaddingLayoutNode ["*",10,"*",10] # vertically center, pad with 10px
-RowsLayoutNode # display some rectangles in rows
--HintedLayoutNode [100,100] # red square
---BackgroundNode [255,0,0,255]
---HintedLayoutNode [30,30] # mini black square
----BackgroundNode [0,0,0,255]
--HintedLayoutNode [50,150] # blue rectangle
---BackgroundNode [0,0,255,255]
--BackgroundNode [0,0,0,255] # green rectangle with black border
---PaddingLayoutNode [5,5,5,5] 
----HintedLayoutNode [150,50]
-----BackgroundNode [0,255,0,255]
""")
keys["cols"].add_child(first_column)




In [4]:
# second example: a block layout of images
second_column, second_keys = deserialize_node("""
HintedLayoutNode [100,"*"] # width of 100px
-BackgroundNode [100,100,100,255] # gray background
--blocklayout BlocksLayoutNode # place as many children as you can every row
""")
keys["cols"].add_child(second_column)

# put the animations in the spritesheet into AnimatedNodes, and put those into the BlockLayoutNode
for key in sprites.keys():
    node = AnimatedNode()
    node.animation = sprites[key]
    second_keys["blocklayout"].add_child(node)


In [5]:
# third example: some custom drawing
import math

def my_listener():
    _, x,y,width,height = yield "on_layout"
    
    t = 0
    
    while True:
        event, *args = yield ["on_draw","on_layout","on_frame"]
        
        if event == "on_layout":
            x,y,width,height = args
        
        if event == "on_frame":
            dt = args[0]
            t += dt
            
            # prevent overflow
            while t > math.pi*2:
                t -= math.pi*2
        
        if event == "on_draw":
            glColor4f(1,0,0,1) # red
            
            # draw a rectangle at sin(t)
            
            rectheight = 30
            ypos = y+(height-rectheight)* (math.sin(t) + 1)/2 
            
            glRectf(x,ypos,x+width,ypos+rectheight)



third_column, third_keys = deserialize_node("""
HintedLayoutNode [100,"*"] # width of 200px
-PaddingLayoutNode [0,"*",0,"*"] # vertically center contents
--listenernode ListenerNode # custom drawing
""")

keys["cols"].add_child(third_column)

third_keys["listenernode"].listener = my_listener
# a listenernode listener gets the same events as a window listener
# except it gets 'on_draw' when the node gets drawn, not the window
# and it also gets the 'on_layout' event which returns x,y,width,height
   

window.node, _ = deserialize_node("""
HintedLayoutNode [1920,1080]
-ListenerNode # my game is drawn here
""") 
    
menu_node,_ = deserialize_node("""
-PaddingLayoutNode [5,5,5,5]
--ColsLayoutNode
---LabelNode "Health potion"
---LabelNode "Stength potion"
""")


def pause_listener():
    paused = False
    
    while True:
        _, symbol, _  yield "on_key_press"
            
        if symbol = pyglet.window.keys.SPACE:
            if not paused:
                paused = True
                window.node.add_child(menu_node)
            else:
                paused = False
                window.node.splice(1,1)



            
        
    


In [6]:

# focus system
# dialog system
# displaying text

fourth_column, fourth_keys = deserialize_node("""
PaddingLayoutNode [10,10,"*",10] # pad the bottom and sides
-RowsLayoutNode
--LabelNode "These text boxes are editable!"
--TextBoxNode "Edit me (1)"
--TextBoxNode "Edit me (2)"
--TextBoxNode "Edit me (3)"
--PaddingLayoutNode [10,10,10,10] # button node
---button BackgroundNode [255,0,0,255] 
----LabelNode "Click me"
--label LabelNode "I am text."
""")
keys["cols"].add_child(fourth_column)

# to make the textboxes be able to listen for events you need to
# enable focus events. you might want to disable these if a dialog is open

global dialog_open
dialog_open = False

def focus_events_enabled():
    global dialog_open
    return not dialog_open    

# these are imported from swyne.focus
route_focus_events(w, enabled=focus_events_enabled) # forward the key events
mouse_focus_via_pos_dims(w, enabled=focus_events_enabled) # something gets focus via click



def button_handler(button):
    
    global dialog_open
    
    while True:
        _, x,y,_,_ = yield "on_mouse_press"
        
        if dialog_open: return
        
        # get button position and dimensions
        pos,dims = button.pos, button.dims
        
        # current label text
        current_text = fourth_keys["label"].document.text
        
        if pos.x < x and x < pos.x+dims.x and pos.y < y and y < pos.y+dims.y:
            # launch prompt dialog
            dialog_open = True
            new_text = yield from prompt_dialog(w, "Enter text:", current_text)
            dialog_open = False
            
            # update label text
            if new_text is not None:
                fourth_keys["label"].document.text = new_text
    
    
w.launch_listener(button_handler, fourth_keys["button"])


In [7]:
# features I didnt mention:

# serialize image

# ScrollLayoutNode

In [11]:
try:
    pyglet.app.run()
except KeyboardInterrupt:
    pass

In [10]:
fourth_keys["label"].document.text = "Hi Chris"